In [ ]:
#%%
import sys
import os
sys.path.append("C:\ZSOMBI\OneDrive\PoPy")

import popy.config as cfg
from popy.behavior_data_tools import drop_time_fields, add_history_of_feedback, add_switch_info

import pandas as pd

In [ ]:
def behav_processor(session_data):
    session_data = drop_time_fields(session_data)  # drop time fields
    session_data = add_history_of_feedback(session_data, 8, one_column=False)
    session_data = add_switch_info(session_data)  # add switch info
    session_data['target-1'] = session_data['target'].shift(1)  # add target-1
    return session_data

In [ ]:
#%%
base_path = cfg.PROJECT_PATH_LOCAL
out_path = os.path.join(base_path, 'data', 'processed', 'behavior')

# read fnames in dir
fnames = os.listdir(out_path)

df_concat = pd.DataFrame()

# load files one by one and process them
for f in fnames:
        monkey, session = f.split('_')[:2]
        session_data = pd.read_pickle(out_path + '\\' + f)  # load data
        # add session info (monkey, session)
        session_data['monkey'] = monkey
        session_data['session'] = session
        # process data (add and drop columns)  --  only modify this function to customize concatenation
        session_data = behav_processor(session_data)
        # concatenate
        df_concat = pd.concat([df_concat, session_data], ignore_index=True)

In [ ]:
# rearrange columns: monkey, session, trialid, blockid, best target, target, feedback, switch, history of feedback
df_concat = df_concat[['monkey', 'session', 'trial_id', 'block_id', 'best_target', 'target', 'target-1', 'feedback', 'switch', 'R-1', 'R-2', 'R-3', 'R-4', 'R-5', 'R-6', 'R-7', 'R-8']]

df_concat

In [ ]:
# save concatenated data
df_concat.to_pickle(out_path + '\\concatenated_behavior_data.pkl')